<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/baseline_norag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
transformers 4.57.1 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is inc

In [ ]:
# set parameter
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"
data_path = "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp" # folder path containing ewah.pdf and samples.csv

In [ ]:
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatUpstage(api_key = UPSTAGE_API_KEY)

In [ ]:
prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer for the question below.
    Write the answer in form: "[ANSWER] (Answer Alphabet) substance"
    for example, the answer can be "[ANSWER]: (A) convolutional networks"
    ---
    Question: {question}
    ---
    """
)

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/testset.csv")

In [ ]:
chain = LLMChain(
    llm = llm,
    prompt=prompt_template
)

In [ ]:
def get_answer(question:str) -> str:
  result = chain.run(question = question)
  return result.strip()

In [ ]:
df["model_answer"] = df["prompts"].apply(get_answer)

In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/results_with_model_answer.csv", index=False)
df.head()

,prompts,answers,model_answer
0,QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 ...,(D),[ANSWER]: (C) 60일 (Answer Alphabet) substance
1,QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 ...,(A),"[ANSWER]: (D) A,B,C 중 답 없음\n\nExplanation: The..."
2,QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는...,(C),(B) 18학점 (Answer Alphabet) substance
3,QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?\n(...,(D),[ANSWER]: (D) 로라 : 휴학기간 경과 후 3주가 지났으나 갑자기 큰 사고...
4,QUESTION5) 2019학년도 휴먼기계바이오공학부의 입학 정원은 몇 명인가? \...,(C),[ANSWER]: (C) 110명 substance


In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/results_with_model_answer.csv", index=False)

In [ ]:
answer_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/results_with_model_answer.csv")

In [ ]:
import re

def extract(s:str) -> str:
  if pd.isna(s):
    return None
  s = str(s)
  m = re.search(r"\(([A-Z])\)", s)
  if m:
    return m.group(1)
  return None

In [ ]:
answer_df["answer_alphabet"] = answer_df["answers"][:26].apply(extract)
answer_df["model_answer_alphabet"] = answer_df["model_answer"][:26].apply(extract)

answer_df["correct"] = (answer_df["answer_alphabet"][:26] == answer_df["model_answer_alphabet"][:26]).astype(int)
accuracy = answer_df["correct"].mean()
print(f"Solar without rag accuracy: {accuracy * 100:.2f}%")

Solar without rag accuracy: 61.54%
